In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics.cluster import contingency_matrix 



In [2]:
%store -r test

# Take the name of the different classes (e.g. '0', '1', '2', ...)
classes = test.groupby(['label'])['label'].unique().index.to_list()

In [3]:
predicted_test = test.copy()

feature_1= 'curvature_2'


models = []
for n_class in classes: # Can be thrown an expection beacuse this is the labels of the test set and not of the models (train set)
    model = pickle.load(open('models_3/'+str(n_class)+'hmm_model.pkl', 'rb'))
    models.append(model)

#log_likelihoods = pd.DataFrame()
#log_likelihoods['predicted_class'] = classes
#log_like_max = []
#top_3_loglikelihood = []
top_3_predicted_classes = []
predicted_class = []
#is_predicted = []
true_label=[]

for i in range(len(test)):
    angle_1 = test[feature_1].iloc[i]
    X=np.array([angle_1]).T
    true_label.append(test['label'].iloc[i])
    list_log_likelihoods = []
    for model in models:
        
        list_log_likelihoods.append(model.score(X,lengths=len(X)))
        pass

    predicted_class.append(np.argmax(list_log_likelihoods))

    top_3_values = sorted(list_log_likelihoods)[-3:]
    top_3_predicted_classes.append([i for i, x in enumerate(list_log_likelihoods) if x == top_3_values[0] or x == top_3_values[1] or x == top_3_values[2]])
    

In [4]:
matrix=contingency_matrix(true_label, predicted_class)

In [5]:
matrix


array([[18,  0,  0,  0,  0,  0, 10,  1,  5,  0],
       [ 0,  2,  1,  1,  9,  0,  0,  7,  1,  2],
       [ 0,  1,  7,  2,  1,  3,  1,  0,  3,  5],
       [ 0,  1,  3, 10,  1,  1,  5,  0,  0,  1],
       [ 0,  1,  0,  0, 20,  0,  0,  4,  0,  0],
       [ 1,  0,  1,  2,  1, 12,  2,  1,  3,  2],
       [11,  2,  1,  0,  0,  0,  7,  0,  4,  0],
       [ 0,  3,  0,  0,  4,  4,  0,  9,  1,  0],
       [ 3,  0,  1,  1,  0,  0,  4,  1, 14,  0],
       [ 0,  0,  3,  4,  1,  1,  3,  0,  1, 13]])

In [6]:
accuracy=np.sum(np.diag(matrix))/np.sum(matrix)
print("Accuracy: ", accuracy)

Accuracy:  0.45161290322580644


In [7]:
# Calculate top 3 accuracy
sum_top_3 = 0
for i in range(len(top_3_predicted_classes)):
    if true_label[i] in top_3_predicted_classes[i]:
        sum_top_3 += 1

accuracy_top_3 = sum_top_3/len(predicted_test["label"])
print("Accuracy top 3: ", accuracy_top_3)

Accuracy top 3:  0.8225806451612904
